
The IMDb Datasets is an open source set of data that is taken from [the IMDb website](https://www.imdb.com/interfaces/), and is available to do some interesting analyses on.

## Step 1

There are a lot of interesting things that we can glean from this dataset. First, we should download the titles and ratings datasets.

In [ ]:
import os

!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

!wget https://datasets.imdbws.com/title.akas.tsv.gz
!gunzip title.akas.tsv.gz

!wget https://datasets.imdbws.com/title.crew.tsv.gz
!gunzip title.crew.tsv.gz

!wget https://datasets.imdbws.com/title.ratings.tsv.gz
!gunzip title.ratings.tsv.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=26dd9610ac4813d3ed9333224b63da9ed263e027d1aaa4b944e67623f769a7b4
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 23 not

In [ ]:
#@title Create a Spark Session and Context

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# Step 2

Read in the datasets

In [ ]:
!head -n 10000 title.akas.tsv > akas.short.tsv
!head -n 10000 title.crew.tsv > crew.short.tsv
!head -n 10000 title.ratings.tsv > ratings.short.tsv

In [ ]:
"/content/crew.short.tsv"

titles = spark.read.csv('akas.short.tsv', sep=r'\t', header=True)
ratings = spark.read.csv('ratings.short.tsv', sep=r'\t', header=True)
crews = spark.read.csv('crew.short.tsv', sep=r'\t', header=True)

# Join the Ratings and the Crew Information

In [ ]:
# titles = spark.read.csv('title.akas.tsv', sep=r'\t', header=True)

titles.show()
ratings.show()
crews.show()

+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|  titleId|ordering|               title|region|language|      types|          attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|tt0000001|       1|          Карменсіта|    UA|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       2|          Carmencita|    DE|      \N|         \N|       literal title|              0|
|tt0000001|       3|Carmencita - span...|    HU|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       4|          Καρμενσίτα|    GR|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       5|          Карменсита|    RU|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       6|          Carmencita|    US|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       7|          Carmenc

DataFrame[tconst: string, directors: string, writers: string, averageRating: string, numVotes: string]

In [ ]:
crews.join(ratings, "tconst", how="outer").show(200 join)

+---------+-------------------+---------+-------------+--------+
|   tconst|          directors|  writers|averageRating|numVotes|
+---------+-------------------+---------+-------------+--------+
|tt0000001|          nm0005690|       \N|          5.7|    1944|
|tt0000002|          nm0721526|       \N|          5.8|     263|
|tt0000003|          nm0721526|       \N|          6.5|    1769|
|tt0000004|          nm0721526|       \N|          5.6|     179|
|tt0000005|          nm0005690|       \N|          6.2|    2579|
|tt0000006|          nm0005690|       \N|          5.1|     177|
|tt0000007|nm0005690,nm0374658|       \N|          5.4|     809|
|tt0000008|          nm0005690|       \N|          5.4|    2088|
|tt0000009|          nm0085156|nm0085156|          5.3|     203|
|tt0000010|          nm0525910|       \N|          6.9|    7036|
|tt0000011|          nm0804434|       \N|          5.3|     362|
|tt0000012|nm0525908,nm0525910|       \N|          7.4|   12065|
|tt0000013|          nm05